In [10]:
# Importing all necessary packages at the start of the notebook
import pandas as pd
import scipy.stats as stats
from matplotlib import pyplot as plt
%matplotlib inline
import urllib.request
from os.path import exists
from sklearn.preprocessing import MinMaxScaler
from helper_functions import *

In [11]:
url = 'https://github.com/MicrosoftDocs/ml-basics/raw/master/challenges/data/flights.csv'
filename = 'dataset/explore/flights.csv'
if not exists(filename):
    urllib.request.urlretrieve(url, filename)
else:
    print(f'{filename} file already present')

dataset/explore/flights.csv file already present


In [12]:
df = pd.read_csv(filename)

In [13]:
df.isnull().any()

Year                 False
Month                False
DayofMonth           False
DayOfWeek            False
Carrier              False
OriginAirportID      False
OriginAirportName    False
OriginCity           False
OriginState          False
DestAirportID        False
DestAirportName      False
DestCity             False
DestState            False
CRSDepTime           False
DepDelay             False
DepDel15              True
CRSArrTime           False
ArrDelay             False
ArrDel15             False
Cancelled            False
dtype: bool

In [ ]:
# Since DepDel15 contains null values, we will try to impute the values from that column
